## SmileDetector
Building a Smile Detection System Using Computer Vision.

Explore the fascinating world of computer vision by creating a system that can detect and recognize smiles. \
Task 1: Create a smile detector using OpenCV\
Task 2: Using a Convolutional Neural Network (CNN), you will learn how to collect data, preprocess images, train a model, and deploy it for smile detection.

## Task 1

### Step 1:Include the desired haar-cascades.
Haar-cascades are classifiers that are used to detect features (of face in this case) by superimposing predefined patterns over face segments and are used as XML files. You can use to detect face, eye, smiles, etc. In our model, we will use face haar-cascades.


In [ ]:
import cv2
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades +'haarcascade_frontalface_default.xml')


### Step 2: define the detect_faces function

In [ ]:
# Function to detect smiles in an image
def detect_faces(image_path):
    # Read the image
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Detect faces in the image
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        # Draw rectangle around face
        cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]

    return(img)

uploaded = files.upload()
file_name = next(iter(uploaded))
img=detect_faces(file_name)

In [ ]:
### display detected faces
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
import matplotlib.pyplot as plt
plt.figure(figsize=(20,10))
plt.imshow(img_rgb)
plt.axis('off')

## Task 2
### Step 1: data collection
we can download [smile or not face](https://www.kaggle.com/datasets/chazzer/smiling-or-not-face-data/data) data from Kaggle for training.
1. First, go to Kaggle and make sure that you're logged in. Then go to your profile pic in the top right, and click on it. Then select Your Profile, then select the Account tab, and finally scroll down to the API section. Click Create New Token. download kaggle.json and upload it to Colab through the following code:

In [ ]:
! pip install -q kaggle
from google.colab import files
files.upload()

2. we'll download the data.

In [ ]:
# make kaggle directory
!mkdir ~/.kaggle
# move kaggle.json to hidden kaggle folder
!cp kaggle.json ~/.kaggle/
# change permissions on file
!chmod 600 ~/.kaggle/kaggle.json
# download zipped data
!kaggle datasets download -d chazzer/smiling-or-not-face-data


3. unzip the Data. The -qq just tells the computer not to print out the name of each file as it unzips.

In [ ]:
!unzip -qq smiling-or-not-face-data.zip

## generate a new folder smile_detector and move all data to smile_detector

In [ ]:
import shutil
import os
# Define source and destination paths
source_folder1 = './non_smile'
source_folder2 = './smile'
source_folder3 = './test'
destination_folder_train = './smile_detector/train/'  # Moving to 'new_location' directory
destination_folder_test = './smile_detector'  # Moving to 'new_location' directory
# Ensure the destination folder exists
os.makedirs(os.path.dirname(destination_folder_train), exist_ok=True)
try:
    shutil.move(source_folder1, destination_folder_train)
    shutil.move(source_folder2, destination_folder_train)
    shutil.move(source_folder3, destination_folder_test)
    print("Folders moved successfully.")
except FileNotFoundError as e:
    print(f"Error: {e}")
except shutil.Error as e:
    print(f"Error: {e}")


Folders moved successfully.


In [ ]:
train_dir = "/content/smile_detector/train"
test_dir = "/content/smile_detector/test"

Now use image_dataset_from_directory function to read your training data

In [ ]:
### start your code here
from tensorflow.keras.utils import image_dataset_from_directory

train_ds = image_dataset_from_directory(train_dir, image_size=(64, 64), batch_size=32)


NameError: name 'train_dir' is not defined

In [ ]:
class_names=train_ds.class_names

### design and build your own CNN model

In [ ]:
### start your code here
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import Adam


model = Sequential()
#
model.add(Conv2D(filters = 8, kernel_size = (5,5),padding = 'Same',
                 activation ='relu', input_shape = (64,64,3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))
#
model.add(Conv2D(filters = 16, kernel_size = (3,3),padding = 'Same',
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))
# fully connected
model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation = "sigmoid"))

Compile model

In [ ]:
optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

Train your model

In [ ]:
history = model.fit(
    train_ds,
    epochs=20
)

Epoch 1/20
38/38 [==============================] - 8s 134ms/step - loss: 13.3995 - accuracy: 0.5062
Epoch 2/20
38/38 [==============================] - 5s 117ms/step - loss: 0.7571 - accuracy: 0.5220
Epoch 3/20
38/38 [==============================] - 4s 102ms/step - loss: 0.7091 - accuracy: 0.5337
Epoch 4/20
38/38 [==============================] - 5s 120ms/step - loss: 0.6787 - accuracy: 0.5686
Epoch 5/20
38/38 [==============================] - 5s 126ms/step - loss: 0.6893 - accuracy: 0.5478
Epoch 6/20
38/38 [==============================] - 4s 102ms/step - loss: 0.6683 - accuracy: 0.5819
Epoch 7/20
38/38 [==============================] - 5s 122ms/step - loss: 0.6379 - accuracy: 0.6027
Epoch 8/20
38/38 [==============================] - 5s 119ms/step - loss: 0.5763 - accuracy: 0.6941
Epoch 9/20
38/38 [==============================] - 4s 103ms/step - loss: 0.5102 - accuracy: 0.7648
Epoch 10/20
38/38 [==============================] - 4s 117ms/step - loss: 0.4611 - accuracy: 0.793

Upload a new image, either yourself or find a face pic online

In [ ]:
uploaded = files.upload()
file_name = next(iter(uploaded))

Saving James_Jones_0001.jpg to James_Jones_0001.jpg


Make prediction

In [ ]:
import numpy as np
img = cv2.resize(cv2.imread(file_name),(64,64))
img = np.reshape(img,[1,64,64,3])
classes = model.predict(img)
classes = np.argmax(classes )
class_names[classes]

1/1 [==============================] - 0s 22ms/step


'non_smile'